In [1]:
%pylab inline

from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timedelta

from dateutil.parser import parse

Populating the interactive namespace from numpy and matplotlib


In [67]:
parse('24th sept  1992')

datetime.datetime(1992, 9, 24, 0, 0)

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

In [90]:
def get_temperature(fromMonth, fromDay, fromYear, toMonth, toDay, toYear, station):    
    target_url = f"http://www.georgiaweather.net/index.php?variable=HI&site={station}"

    html = requests.post(target_url, data=[('fromMonth',fromMonth),
                                            ('fromDay',fromDay),
                                            ('fromYear', fromYear),
                                            ('toMonth', toMonth),
                                            ('toDay', toDay),
                                            ('toYear', toYear)
                                           ]).text

    soup = BeautifulSoup(html)
    res = soup.find_all('table')
    
    #Processing
    parser = HTMLTableParser()
    df = parser.parse_html_table(tables[1])
    df = df.iloc[6:,:4]                  
    df.columns = df.iloc[0]
    df = df.drop(6).reset_index(drop=True)
    df['Max Temperature[&degF]'] = df['Max Temperature[&degF]'].apply(lambda x : float(x))
    df['Min Temperature[&degF]'] = df['Min Temperature[&degF]'].apply(lambda x : float(x))
    df['Rain (in)'] = df['Rain (in)'].apply(lambda x : float(x))

    #Date processing
    datelist = df['Date']
    date_result_list = []
    for idx, datestring in enumerate(datelist):
        date_time = datetime.strptime(datelist[0] + f' {fromYear}', '%b %d %Y') + timedelta(idx)
        date_result_list.append(date_time)
    df['Date'] = pd.Series(date_result_list)
    
    return df

df = get_temperature('August', 1, 2018, 'August', 31, 2018, 'WATUGA')

In [91]:
df

6,Date,Max Temperature[&degF],Min Temperature[&degF],Rain (in)
0,2018-08-01,78.9,69.2,1.33
1,2018-08-02,73.6,69.9,0.86
2,2018-08-03,85.7,70.3,0.81
3,2018-08-04,88.6,69.1,0.00
4,2018-08-05,91.6,70.7,0.33
5,2018-08-06,92.6,70.5,0.00
6,2018-08-07,90.6,71.6,0.00
7,2018-08-08,89.7,70.2,0.00
8,2018-08-09,89.1,69.5,0.26
9,2018-08-10,87.4,69.1,0.01


In [87]:
from datetime import datetime


0    2016-08-01
1    2016-08-02
2    2016-08-03
3    2016-08-04
4    2016-08-05
5    2016-08-06
6    2016-08-07
7    2016-08-08
8    2016-08-09
9    2016-08-10
10   2016-08-11
11   2016-08-12
12   2016-08-13
13   2016-08-14
14   2016-08-15
15   2016-08-16
16   2016-08-17
17   2016-08-18
18   2016-08-19
19   2016-08-20
20   2016-08-21
21   2016-08-22
22   2016-08-23
23   2016-08-24
24   2016-08-25
25   2016-08-26
26   2016-08-27
27   2016-08-28
28   2016-08-29
29   2016-08-30
30   2016-08-31
dtype: datetime64[ns]

In [82]:

timedelta(4)

datetime.timedelta(4)